In [ ]:
pip install transformers datasets torchaudio jiwer librosa evaluate

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from datasets import load_dataset, DatasetDict

dataset = DatasetDict()
dataset["train"] = load_dataset("/content/drive/My Drive/dataset_new", split='train')  # Works in Colab

print(dataset)

In [ ]:
# from datasets import load_dataset, DatasetDict

# dataset = DatasetDict()
# dataset["train"] = load_dataset("itskavya/gp", split='train') # USE THIS ON YOUR MACHINE
# print(dataset)

In [ ]:
# Get column names to identify the second column
column_names = dataset["train"].column_names
second_column = column_names[1]  # Get the second column name
second_column

In [ ]:
import re

# Remove only punctuation & numbers, but keep Arabic dialect words
chars_to_remove_regex =  '[\,\?\.\!\;\:\؟\"\“\%\‘\”\�\.\،\-\']'

def clean_text(batch):
    batch["transcription"] = re.sub(chars_to_remove_regex, '', batch["transcription"]).lower().strip() # lower so vocab doesn't need capital chars also
    return batch

dataset = dataset.map(clean_text)

In [ ]:
import re

# filter rows that have anything other than letters
def filter_dataset_2(dataset):
    filtered_rows = []
    for i in range(len(dataset['transcription'])):
        if bool(re.search(r'[^A-Za-z\u0621-\u064A\s]', dataset['transcription'][i])):
            filtered_rows.append(i)

    return dataset.select(filtered_rows)

filtered_dataset2 = filter_dataset_2(dataset['train'])

print(filtered_dataset2)

In [ ]:
for i in range(len(filtered_dataset2)):
  print(filtered_dataset2[i]['audio']['path'])
  print(filtered_dataset2[i]['transcription'])
  print("\n")

In [ ]:
import re

# filter rows that have both eng and arabic
def filter_dataset_3(dataset):
    filtered_rows = []
    for i in range(len(dataset['transcription'])):
        if bool(re.search(r'[A-Za-z]', dataset['transcription'][i]) and re.search(r'[\u0600-\u06FF]', dataset['transcription'][i])):
            filtered_rows.append(i)

    return dataset.select(filtered_rows)

filtered_dataset3 = filter_dataset_3(dataset['train'])

print(filtered_dataset3)

In [ ]:
for i in range(len(filtered_dataset3)):
  print(filtered_dataset3[i]['audio']['path'])
  print(filtered_dataset3[i]['transcription'])
  print("\n")

In [ ]:
!pip install PyArabic

In [ ]:
import pyarabic.araby as araby

# remove diacritic
def remove_diacritic(example):
    example['transcription'] = araby.strip_diacritics(example['transcription'])
    return example

dataset['train'] = dataset['train'].map(remove_diacritic)
print(dataset)

In [ ]:
filtered_dataset2 = filter_dataset_2(dataset['train'])

print(filtered_dataset2)

for i in range(len(filtered_dataset2)):
  print(filtered_dataset2[i]['audio']['path'])
  print(filtered_dataset2[i]['transcription'])
  print("\n")

In [ ]:
# split the dataset for testing n validation

dataset = dataset["train"].train_test_split(test_size=0.3)
dataset

In [ ]:
my_dataset_test = dataset['test'].train_test_split(test_size=0.5)
print(my_dataset_test)

In [ ]:
dataset['validation'] = my_dataset_test['train']
dataset['test'] = my_dataset_test['test']
print(dataset)

In [ ]:
# this is not giving eng chars or all arabic chars in the vocab

# def extract_all_chars(batch):
#     all_texts = [" ".join(text) for text in batch["transcription"]]
#     print(all_texts)
#     vocab_list = [list(set(text)) for text in all_texts]
#     print(vocab_list)
#     return {
#         "vocab": vocab_list,
#         "all_text": all_texts
#     }

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  print(all_text)
  vocab = list(set(all_text))
  print(vocab)
  return {"vocab": [vocab], "all_text": [all_text]}


In [ ]:
vocab_train = dataset['train'].map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset['train'].column_names)
vocab_val = dataset['validation'].map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset['validation'].column_names)
vocab_test = dataset['test'].map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset['test'].column_names)

In [ ]:
# all distinct letters in the training dataset and test dataset

vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0])| set(vocab_val["vocab"][0]))

In [ ]:
print(len(vocab_list))

In [ ]:
print((vocab_list))

In [ ]:
vocab_list.append('q')
vocab_list.append('z')

In [ ]:
vocab_dict = {k:v for k, v in enumerate(sorted(vocab_list))}
vocab_dict

In [ ]:
vocab_dict2 = {}
for k, v in vocab_dict.items():
  vocab_dict2[v] = k
print(vocab_dict2)

In [ ]:
# let space be | and remove new line

vocab_dict = vocab_dict2
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
del vocab_dict["\n"]

In [ ]:
vocab_dict

In [ ]:
len(vocab_dict)

In [ ]:
# add unk and padding tokens

vocab_dict["[UNK]"] = len(vocab_dict) + 1 # plus one because the last char already has len as value
vocab_dict["[PAD]"] = len(vocab_dict) + 1
len(vocab_dict)

In [ ]:
vocab_dict

In [ ]:
# save the vocab

import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|") # load vocab to the tokenizer

In [ ]:
repo_name = "wav2vec2-arabic-colab"

In [ ]:
tokenizer.push_to_hub(repo_name)

In [ ]:
input_str = filtered_dataset2[0]["transcription"]
labels = tokenizer(input_str).input_ids # it returns a dict of input ids and attention mask so just get the input ids
print(labels)
print(tokenizer.tokenize(input_str))
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True, group_tokens=False)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
# the feature extractor and tokenizer are wrapped into a single Wav2Vec2Processor class so that one only needs a model and processor object

from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
#check the sampling rate

dataset['train'][0]["audio"]

In [ ]:
# set the audio feature to the correct sampling rate

from datasets import Audio

dataset['train'] = dataset['train'].cast_column("audio", Audio(sampling_rate=16_000))
dataset['validation'] = dataset['validation'].cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
dataset['train'][0]["audio"]

In [ ]:
#check that the data is correctly prepared

import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(dataset['train'])-1)

print(dataset['train'][rand_int]["transcription"])
ipd.Audio(data=dataset['train'][rand_int]["audio"]["array"], autoplay=True, rate=16000)

In [ ]:
rand_int = random.randint(0, len(dataset['train'])-1)

print("Target text:", dataset['train'][rand_int]["transcription"])
print("Input array shape:", dataset['train'][rand_int]["audio"]["array"].shape)
print("Sampling rate:", dataset['train'][rand_int]["audio"]["sampling_rate"])

In [ ]:
#apply the data preparation function to all examples

def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

In [ ]:
dataset['train'] = dataset['train'].map(prepare_dataset, remove_columns=dataset['train'].column_names)
dataset['validation'] = dataset['validation'].map(prepare_dataset, remove_columns=dataset['validation'].column_names)

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
import evaluate
metric = evaluate.load("wer")
metric2 = evaluate.load("cer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = metric.compute(predictions=pred_str, references=label_str)
    cer = 100 * metric2.compute(predictions=pred_str, references=label_str)

    return {"wer": wer, "cer":cer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

In [ ]:
model.freeze_feature_extractor()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()